In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
#!pip install langchain langchain-community faiss-cpu PyPDF2 sentence-transformers

In [24]:
#!pip install ollama

In [25]:
import os
import glob
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from typing import List
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

pdf_directory = "./pdf"
txt_directory = "./txt"

documents = []

pdf_files = glob.glob(os.path.join(pdf_directory, "*.pdf"))
print(f"{len(pdf_files)}개의 PDF 파일을 발견했습니다.")

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

txt_files = glob.glob(os.path.join(txt_directory, "*.txt"))
print(f"{len(txt_files)}개의 TXT 파일을 발견했습니다.")

for txt_file in txt_files:
    loader = TextLoader(txt_file, encoding='utf-8')  # 한글 인코딩 지원
    documents.extend(loader.load())
print(f"총 {len(documents)}개의 문서 로드 완료")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)
print(f"총 {len(chunks)}개의 청크로 분할 완료")

46개의 PDF 파일을 발견했습니다.
6개의 TXT 파일을 발견했습니다.
총 368개의 문서 로드 완료
총 596개의 청크로 분할 완료


In [26]:
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},     # GPU 있으면 'cuda'로 변경
    encode_kwargs={'normalize_embeddings': True}
)

In [27]:
# FAISS 벡터 저장소 생성
vector_store = FAISS.from_documents(chunks, embeddings)

# 벡터 저장소 저장 (나중에 재사용할 수 있도록)
vector_store.save_local("faiss_index")
print("벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.")

# Ollama 모델 설정 (gemma3:1b 사용)
llm = Ollama(model="gemma3:1b")

벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.


In [ ]:
# 항공 금지 품목 관련 시스템 프롬프트
SYSTEM_PROMPT = """
당신은 여행 금지 품목 및 제한사항에 대한 전문적인 지식을 갖춘 챗봇입니다.
각국의 입국 시 금지/제한되는 품목에 대해 정확한 정보를 제공해야 합니다.
사용자가 질문한 내용과 관련하여 검색된 문서의 정보를 참고하여 답변하되,
검색된 문서에 관련 정보가 없으면 일반적인 지식을 바탕으로 답변하세요.
사용자가 나라를 명시하지 않으면 추가 정보를 요청하세요.
알 수 없는 정보에 대해서는 솔직하게 모른다고 답변하고, 공식 정보를 확인할 것을 권장하세요.

주요 국가별 일반적인 금지/제한 품목:
- 일본: 마약류, 총기류, 가짜 상품, 특정 농산물과 육류
- 미국: 과일, 채소, 육류, 식물, 특정 의약품, 6개월 이상된 식품
- 중국: 정치적으로 민감한 자료, 종교 관련 자료, 마약류, 도박 관련 물품
- 태국: 마약류, 담배(200개비 초과), 술(1리터 초과), 일부 의약품
- 호주: 신선 과일, 채소, 육류, 유제품, 씨앗, 목재 제품

항공기 탑승 시 일반적 제한:
- 액체류: 개별 용기 100ml 이하, 1L 투명 지퍼백에 담아야 함
- 예외: 유아식, 의약품 등(보안 검색 시 별도 신고)
- 위험물: 라이터(1개만 기내 반입 가능), 성냥, 스프레이, 압축 가스
- 날카로운 물건: 칼, 가위, 면도칼 등은 위탁 수하물로만 가능
"""

# 프롬프트 템플릿 정의 - 수정된 부분
prompt_template = """
{system_prompt}

질문: {question}

다음은 질문과 관련된 문서 정보입니다:
{context}

답변:
"""

PROMPT = PromptTemplate(
    input_variables=["system_prompt", "question", "context"],
    template=prompt_template
)

# LangChain 체인 생성 - 수정된 부분
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": True
    },
    return_source_documents=True
)

# qa_chain 실행 방법 (필요시 사용)
def ask_with_qa_chain(query: str) -> str:
    """
    LangChain의 RetrievalQA를 사용하여 질의응답
    """
    result = qa_chain({"question": query, "system_prompt": SYSTEM_PROMPT})
    return result["result"]

# 직접 Ollama API 사용하여 질의응답하는 함수
def ask_with_ollama(query: str, retrieved_docs: List[str]) -> str:
    """
    Ollama API를 직접 사용하여 질의응답
    
    Args:
        query: 사용자 질문
        retrieved_docs: 검색된 관련 문서 목록
        
    Returns:
        응답 텍스트
    """
    # 검색된 문서들을 하나의 문자열로 합치기
    context = "\n\n".join(retrieved_docs)
    
    # Ollama API 호출을 위한 메시지 준비
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"질문: {query}\n\n다음은 질문과 관련된 문서 정보입니다:\n{context}"
        }
    ]
    
    # Ollama API 호출
    response = ollama.chat(
        model="gemma3:1b",
        messages=messages
    )
    
    return response["message"]["content"]

# 항공 금지 품목 챗봇 기능 구현
class TravelRestrictionsBot:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 3})
        
    def detect_country(self, query: str) -> str:
        """사용자 질문에서 국가명을 감지"""
        countries = ["일본", "미국", "중국", "태국", "호주", "유럽", "한국", 
                    "영국", "프랑스", "독일", "이탈리아", "스페인", "캐나다", 
                    "싱가포르", "말레이시아", "인도네시아", "베트남", "필리핀"]
        
        for country in countries:
            if country in query:
                return country
        
        return ""
    
    def detect_item(self, query: str) -> str:
        """사용자 질문에서 물품명을 감지"""
        items = ["액체", "음식", "과일", "육류", "약", "의약품", "주류", "술", 
                "담배", "화장품", "식품", "전자제품", "배터리", "칼", "가위", "면도기"]
        
        for item in items:
            if item in query:
                return item
        
        return ""
    
    def enhance_query(self, query: str) -> str:
        """사용자 질문 강화"""
        country = self.detect_country(query)
        item = self.detect_item(query)
        
        if country and item:
            return f"{query} {country} {item} 반입 제한 규정"
        elif country:
            return f"{query} {country} 입국 금지 품목"
        elif item:
            return f"{query} {item} 항공 반입 제한"
        else:
            return query
    
    def get_response(self, query: str) -> str:
        """사용자 질문에 대한 응답 생성"""
        # 질문 강화
        enhanced_query = self.enhance_query(query)
        
        # 관련 문서 검색
        docs = self.retriever.get_relevant_documents(enhanced_query)
        doc_contents = [doc.page_content for doc in docs]
        
        # 응답 생성 방식 선택 (두 가지 중 하나 사용)
        # 1. 직접 Ollama API 사용
        response = ask_with_ollama(query, doc_contents)
        
        # 2. LangChain의 RetrievalQA 사용 (주석 처리)
        # response = ask_with_qa_chain(query)
        
        return response

# 챗봇 인스턴스 생성
bot = TravelRestrictionsBot(vector_store)

# 대화형 모드
def chat_mode():
    print("=== 여행 금지 품목 확인 챗봇 ===")
    print("종료하려면 'q' 또는 'exit'를 입력하세요.")
    print("="*50)
    
    while True:
        user_input = input("\n질문을 입력하세요: ")
        
        if user_input.lower() in ['q', 'exit', '종료']:
            print("챗봇을 종료합니다. 감사합니다.")
            break
            
        response = bot.get_response(user_input)
        print(f"\n[챗봇] {response}")

# 실행
if __name__ == "__main__":
    chat_mode()

=== 여행 금지 품목 확인 챗봇 ===
종료하려면 'q' 또는 'exit'를 입력하세요.


/var/folders/kp/5_0zhw410tzf9tkb2lt76sw40000gn/T/ipykernel_11236/2846628453.py:145: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(enhanced_query)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[챗봇] 다음은 질문과 관련된 문서 정보에 따른 답변입니다.

**총 들고 타도 돼?**

*   **제한되는 품목:**
    *   마약류
    *   총기류
    *   가짜 상품
    *   특정 농산물 및 육류
    *   특정 의약품
    *   6개월 이상된 식품
    *   공구류(칼, 가위 등)
    *   기내식 전용 나이프
    *   **전자충격기, 총기, 무술호신용품**
    *   **날카로운 물건**
    *   **여분배터리 100Wh 이하**
    *   **스프레이류**
*   **주의사항:**
    *   항공사 또는 여행사에 위탁 수화물로 총기의 소지가 가능한지 확인해야 함.
    *   적용되는 제한 및 비용을 확인해야 함.

**참고:**

*   **아메리칸항공**의 경우, 총기류, 마약류, 가짜 상품, 특정 농산물, 육류, 의약품, 6개월 이상된 식품, 기내식 전용 나이프, 날카로운 물건, 여분배터리 100Wh 이하 등의 품목에 대한 제한이 있습니다.

**추가 정보:**

*   **TSA 웹사이트 ‘Firearms & Ammunition’**를 참조하여 더 자세한 정보를 확인하시기 바랍니다.
*   **항공사별 제한 사항 및 비용**은 각 항공사에 문의하시는 것이 좋습니다.

[챗봇] 다음은 질문에 대한 답변입니다.

**물 가져가도 돼?**

*   **아니요.** 물은 기내에 반입이 금지됩니다.

**참고:**

*   위 문서에는 물에 대한 구체적인 제한 사항이 명시되어 있지 않습니다.
*   기내에 든 물품은 기내에서 사용하지 않도록 주의해야 합니다.
